In [1]:
import sys
import  datetime
import os
from minio import Minio
import csv
import json
import pandas as pd
import time
import subprocess
from subprocess import Popen, PIPE
import requests
from IPython.display import display
import datetime

In [2]:
PROMETHEUS = 'http://172.17.141.197:31022/'
pre_url = PROMETHEUS + '/api/v1/query?query='
df_metric = pd.DataFrame(columns=['timestamp','pod_count_aeneas','pod_count_tocloud','pod_count_getobject'],index=[0])
df_metric.to_csv('pod_count_resources.csv')
# timestamp= pd.to_datetime(df['outtime'].iloc[-1], format='%Y-%m-%d %H:%M:%S')
# time = (time.mktime(timestamp.timetuple()))
#df['timestamp']=datetime.datetime.now()
df_metrics=pd.read_csv('pod_count_resources.csv')


In [3]:
def getdataprometheus(url):
    headers= {"Accept": "application/json"}
    res = json.loads(requests.post(url=url, headers=headers).content.decode('utf8', 'ignore'))
    data=res.get('data').get('result')[0].get('value')[1]
    return data

In [4]:
df=pd.read_csv('pod_count_resources.csv')
expr_pod_aeneas_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"aeneas.*"})'
expr_pod_tocloud_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"scaling.*"})'
expr_pod_getobject_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"getobject.*"})'
df['pod_count_aeneas']=getdataprometheus(pre_url+expr_pod_aeneas_function)
df['pod_count_tocloud']=getdataprometheus(pre_url+expr_pod_tocloud_function)
df['pod_count_getobject']=getdataprometheus(pre_url+expr_pod_getobject_function)
df['timestamp']=datetime.datetime.now()


In [5]:
expr_pod_cpu_aeneas_function = 'sum((container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn",pod=~"aeneas.*"}))'
expr_pod_cpu_tocloud_function = 'sum((container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn",pod=~"scaling.*"}))'
expr_pod_cpu_mosquitto_broker = 'sum((container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"emqx.*"}))'
expr_pod_cpu_mosquitto_connector_aeneas = 'sum((container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"aeneas.*"}))'
expr_pod_cpu_mosquitto_connector_tocloud = 'sum((container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"tocloud.*"}))'
expr_pod_cpu_functions_all = 'sum((container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}))'

df['pod_cpu_aeneas_function']=getdataprometheus(pre_url+expr_pod_cpu_aeneas_function)
df['pod_cpu_tocloud_function']=getdataprometheus(pre_url+expr_pod_cpu_tocloud_function)
df['pod_cpu_mosquitto_broker']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_broker)
df['pod_cpu_mosquitto_connector_aeneas']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_connector_aeneas)
df['pod_cpu_mosquitto_connector_tocloud']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_connector_tocloud)
df['pod_cpu_functions_all']=getdataprometheus(pre_url+expr_pod_cpu_functions_all)

In [6]:
df_metrics = df_metrics.append(df, ignore_index=True)
df_metrics.to_csv("pod_count_resources.csv",index=False)
display(df_metrics)


/tmp/ipykernel_34906/3860766684.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metrics = df_metrics.append(df, ignore_index=True)


,Unnamed: 0,timestamp,pod_count_aeneas,pod_count_tocloud,pod_count_getobject,pod_cpu_aeneas_function,pod_cpu_tocloud_function,pod_cpu_mosquitto_broker,pod_cpu_mosquitto_connector_aeneas,pod_cpu_mosquitto_connector_tocloud,pod_cpu_functions_all
0,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2022-12-06 06:54:32.189631,2,1,1,552.7162130170001,2220.925530257,21421.918621442,1765.1327257149999,12930.547882878,5837.800462384999
